In [20]:
import os
import sys
import time
import os.path
import csv
import re

In [21]:
def smallest_time(lines):
    for line in lines:
        if "STARTED VOTE" in line:
            time = re.search(r'[0-9]*ms', line).group(0)
            return int(time[:-2])
        elif "STARTED QUERY" in line:
            time = re.search(r'[0-9]*ms', line).group(0)
            return int(time[:-2])

In [39]:
def collect_master():
    for i in range(1, NUM_LOG + 1):
        with open(LOG_ORIGIN + str(i) , "r") as f:
            lines = f.readlines()
            f.close()
    
        num_uncomplete_transactions = 0
        time_cutoff = smallest_time(lines) + 75 * 1000
        time = 0
        with open(TEST + '/output-small'+ '.csv', 'a') as f1:
            with open(TEST + '/backoffcase' + '.csv', 'a') as f2:
                with open(TEST + '/starttime' + '.csv', 'a') as f3:
                    with open(TEST + '/endtime' + '.csv', 'a') as f4:
                        with open(TEST + '/output-big' + '.csv', 'a') as f5:
                            with open(TEST + '/duration_short' + '.csv', 'a') as f6:
                                with open(TEST + '/duratoin_long' + '.csv', 'a') as f7:
                                    writer1 = csv.writer(f1, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    writer2 = csv.writer(f2, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    writer3 = csv.writer(f3, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    writer4 = csv.writer(f4, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    writer5 = csv.writer(f5, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    writer6 = csv.writer(f6, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    writer7 = csv.writer(f7, delimiter=' ', quotechar='|', quoting=csv.QUOTE_MINIMAL)
                                    for line in lines:
                                        if i < BIG_SEP:
                                            if "STARTED VOTE" in line:
                                                time = re.search(r'[0-9]*ms', line).group(0)
                                                time = time[:-2]
                                                num_uncomplete_transactions += 1
                                                writer3.writerow([time])
                                            elif "FINISHED VOTE" in line:
                                                time = re.search(r'IN [0-9]*ms', line).group(0)
                                                time = time[3:-2]
                                                writer6.writerow([time])
                                                time = re.search(r'AT [0-9]*ms', line).group(0)
                                                time = time[3:-2]
                                                num_uncomplete_transactions -= 1
                                                writer4.writerow([time])
                                        else:
                                            if "STARTED QUERY" in line:
                                                time = re.search(r'[0-9]*ms', line).group(0)
                                                time = time[:-2]
                                                num_uncomplete_transactions += 1
                                                writer3.writerow([time])
                                            elif "FINISHED QUERY" in line:
                                                time = re.search(r'IN [0-9]*ms', line).group(0)
                                                time = time[3:-2]
                                                writer7.writerow([time])
                                                time = re.search(r'AT [0-9]*ms', line).group(0)
                                                time = time[3:-2]
                                                num_uncomplete_transactions -= 1
                                                writer4.writerow([time])
                                                
                                        if 0 <= int(time) <= time_cutoff:
                                            attempts = re.search(r'\s[0-9]*\sTXN\sATTEMPTS', line)
                                            if attempts and not '[NOT IN STATS]' in line:
                                                cut = re.search(r'\s[0-9]*\sTXN\sATTEMPTS', line).group(0)
                                                num_attempts = cut.split()[0]
                                                cut = re.search(r'BACKOFFTIME:\s[0-9]*\s', line).group(0)
                                                backofftime = cut.split()[1]
                                                cut = re.search(r'BACKOFFCOUNT:\s[0-9]*\s', line).group(0)
                                                backoffcount = cut.split()[1]
                                                if i < BIG_SEP:
                                                    writer1.writerow([num_attempts, backofftime, backoffcount])
                                                else:
                                                    writer5.writerow([num_attempts, backofftime, backoffcount])
                                            if "becasue" in line:
                                                line = line.strip()
                                                casecode = line.split()[-1]
                                                writer2.writerow([casecode])
                                        else:
                                            break
                                    print num_uncomplete_transactions
                                    f7.close()
                                    f6.close()
                                    f5.close()
                                    f4.close()
                                    f3.close()
                                    f2.close()
                                    f1.close()

In [32]:
TEST = 'var-master-original-new'
LOG_ORIGIN = 'var-master-original-new/log/fabric.log.'
BIG_SEP = 9
NUM_LOG = 10
collect_master()

1
1
1
2
1
2
1
14
1
1


In [33]:
TEST = 'var-master-only-vote-new'
LOG_ORIGIN = 'var-master-only-vote-new/log/fabric.log.'
BIG_SEP = 9
NUM_LOG = 8
collect_master()

1
1
1
1
1
1
1
1


In [34]:
TEST = 'var-master-slow-tally-new'
LOG_ORIGIN = 'var-master-slow-tally-new/log/fabric.log.'
BIG_SEP = 7
NUM_LOG = 8
collect_master()

1
1
1
1
1
1
1
1


In [35]:
TEST = 'var-development-original-new'
LOG_ORIGIN = 'var-development-original-new/log/fabric.log.'
BIG_SEP = 9
NUM_LOG = 10
collect_master()

1
0
18
1
1
8
0
1
0
1


In [36]:
TEST = 'var-development-only-vote-new'
LOG_ORIGIN = 'var-development-only-vote-new/log/fabric.log.'
BIG_SEP = 9
NUM_LOG = 8
collect_master()

1
2
0
1
1
2
0
13


In [37]:
TEST = 'var-development-slow-tally-new'
LOG_ORIGIN = 'var-development-slow-tally-new/log/fabric.log.'
BIG_SEP = 7
NUM_LOG = 8
collect_master()

1
1
1
1
1
1
1
1


In [40]:
TEST = 'var-master'
LOG_ORIGIN = 'var-master/log/fabric.log.'
BIG_SEP = 9
NUM_LOG = 10
collect_master()

0
0
1
7
5
8
13
13
1
1


In [41]:
TEST = 'var-development'
LOG_ORIGIN = 'var-development/log/fabric.log.'
BIG_SEP = 9
NUM_LOG = 10
collect_master()

1
1
2
3
4
11
1
15
1
1
